# Praktikum 2
## Feed Forward Neural Network


___Alvin Sullivan 13515048___

___Albertus Djauhari Djohan 13515054___

___Kevin 13515138___

### Multi Layer Neural Network

#### Implementasi Algoritma Backpropagation



In [47]:
from __future__ import division
import numpy as np
import pandas as pd
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import Dense
from keras import optimizers
from keras.layers import Dropout
from keras.utils import to_categorical
import keras

In [29]:
var_nodes = [18, 17, 16]
var_epoch = 10
var_momentum = 0.001
var_learning_rate = 0.5

#### Eksekusi Dataset

- Membaca dataset
- Praproses Data (Continuous dan Kategorikal)
    - Kategorikal menggunakan StandardScaler

In [41]:
file = "dataset/data_training_praktikum.csv"
data = pd.read_csv(file)

#Handle Categorical Data
le = preprocessing.LabelEncoder()
oh = preprocessing.OneHotEncoder(sparse=False)
data['loc'] = le.fit_transform(data['loc'])
data['struct1'] = le.fit_transform(data['struct1'])
data['struct2'] = le.fit_transform(data['struct2'])
data['struct3'] = le.fit_transform(data['struct3'])
data['cit1'] = le.fit_transform(data['cit1'])
data['cit2'] = le.fit_transform(data['cit2'])
data['labelclass'] = le.fit_transform(data['labelclass'])
data = data.values
X = data[:, 0:-1]
y = data[:, -1]
# y = oh.fit_transform(data[:, -1].reshape(len(data[:, -1]), 1))
# print(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.1, 
                                                    random_state=43)

print('X_train')
print(X_train)

print('X_test')
print(X_test)

print('y_train')
print(y_train)

print('y_test')
print(y_test)

X_train
[[1 0 1 ... 0 0 0]
 [0 1 1 ... 0 0 0]
 [0 1 1 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
X_test
[[0 0 0 ... 0 0 0]
 [0 1 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [1 0 0 ... 0 0 1]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
y_train
[ 8 10 15 ...  7  6 10]
y_test
[10  8  5 ... 10  8 10]


In [ ]:
class MultiLayerNN:

    def __init__(self, data, hidden_node, output_node,
        num_batch, learning_rate_const, tolerance_const,
        **kwargs):
        
        self.instance = data
        self.w_hidden_node = hidden_node
        self.w_output_node = output_node
        # Gradient Descent Parameters
        self.batch_size = num_batch
        self.learning_rate = learning_rate_const
        self.tolerance = tolerance_const
        self.momentum = kwargs.get('momentum', 0)
        self.epochs = kwargs.get('epochs', 10)
       
    def train(self, instance_target):
        instance_target_t = np.array([instance_target]).T
        batch_iteration = int(np.ceil(self.instance.shape[0] / self.batch_size))
        old_loss = -np.inf
        for step in range(self.epochs):
            loss = 0
            for i in range(batch_iteration):
                start_index = i * self.batch_size
                end_index = i * self.batch_size + self.batch_size
                if end_index > len(instance_target_t):
                    end_index = len(instance_target_t)
                o_out = self.gradient_descent(self.instance[start_index:end_index:1], 
                                              instance_target_t[start_index:end_index:1])
                loss = loss + self.loss_function(o_out, 
                                            instance_target_t[start_index:end_index:1])
        # Print Loss
            print ("Loss after epoch %i: %f" % (step, loss/self.instance.shape[0]))
            
            if np.abs(loss - old_loss) < self.tolerance:
                break
            old_loss = loss
    
    def feed_forward(self, instance):
        s = list()
        o = list()
        # Feed Forward Hidden Node
        s_temp = instance.dot(self.w_hidden_node[0].T)
        o_temp = self.sigmoid(s_temp)
        s.append(s_temp)
        o.append(o_temp)
        iteration = len(self.w_hidden_node)
        for i in range(1,iteration):
            s_temp = o[i-1].dot(self.w_hidden_node[i].T)
            o_temp = self.sigmoid(s_temp)
            o.append(o_temp)
        # Feed Forward Output Node
        s_out = o[-1].dot(self.w_output_node.T)
        o_out = self.sigmoid(s_out)
        return s, o, s_out, o_out

    def sigmoid(self, X):
        output = 1 / (1 + np.exp(-X))
        return np.matrix(output)

    def loss_function(self, o_out, instance_target):
        
        squared_error = np.square(instance_target - o_out)
        data_loss = np.sum(squared_error)      
        return data_loss    
    def back_propagation(self, instance_target, o, o_out):
        d = list()
        # Back Propagation Output Node
        d_temp = np.multiply(np.multiply(o_out, 1-o_out), instance_target-o_out)
        d.insert(0, d_temp)
        # Back Propagation Hidden Node
        d_temp = np.multiply(np.multiply(o[-1], 1-o[-1]), 
                             (self.w_output_node.T.dot(d[0].T)).T)
        d.insert(0, d_temp)
        iteration = len(self.w_hidden_node)
        for i in range(iteration-1, 0, -1):
            d_temp = np.multiply(np.multiply(o[i-1], 1-o[i-1]), 
                                 (self.w_hidden_node[i].T.dot(d[0].T)).T)
            d.insert(0, d_temp)
        return d

    def update_weight(self, instance, o, d):
        # Update Weight Output Node
        self.w_output_node[0] = self.w_output_node[0] \
        + self.momentum * self.w_output_node[0] \
        + self.learning_rate * d[-1].T.dot(o[-1])
        # Update Weight Hidden Node
        iteration = len(self.w_hidden_node)
        for i in range(iteration-1, 0, -1):
            self.w_hidden_node[i] = self.w_hidden_node[i] \
            + self.momentum * self.w_hidden_node[i] \
            + self.learning_rate * d[i].T.dot(o[i-1])
        self.w_hidden_node[0] = self.w_hidden_node[0] \
        + self.momentum * self.w_hidden_node[0] \
        + self.learning_rate * d[0].T.dot(instance)
    def gradient_descent(self, instance, instance_target):
        # Feed Forward
        _,o,_,o_out = self.feed_forward(instance)
        # Back Propagation      
        d = self.back_propagation(instance_target, o, o_out)
        # Update Weight
        self.update_weight(instance, o, d)
        return o_out
    def predict(self, instance):
        _,_,s_out,o_out = self.feed_forward(instance)
        return o_out

#### Implementasi Keras



In [42]:
model = Sequential()

for i in range(0,len(var_nodes)):
    if i == 0:
        model.add(Dense(units=var_nodes[i], activation='sigmoid', 
                        input_dim=X_train.shape[1]))
    else:
        model.add(Dense(units=var_nodes[i], activation='sigmoid'))
        
sgd = optimizers.SGD(lr=var_learning_rate, decay=0, 
                     momentum=var_momentum, nesterov=False)

model.compile(optimizer=sgd,
              loss='categorical_crossentropy',
              metrics=['accuracy'])

### Perbandingan Hasil Algoritma Backpropagation dan Keras

#### Inisialisasi bobot awal hidden node dan output node

In [ ]:
np.random.seed(0)
w_hidden_node = list()
for i in range(0,len(var_nodes)-1):
    if i == 0:
        w_temp = np.random.randn(var_nodes[i],
                                 X_train.shape[1]) / np.sqrt(var_nodes[i])
    else:
        w_temp = np.random.randn(var_nodes[i], 
                                 var_nodes[i-1]) / np.sqrt(var_nodes[i])
    w_hidden_node.append(w_temp)

w_output_node = np.random.randn(var_nodes[-1], var_nodes[-2])

print('Matriks weight hidden node:\n', w_hidden_node)
print('Matriks weight output node:\n', w_output_node)

#### Implementasi Mini-Batch (Batch_size = 1)

In [ ]:
var_batch = 1

##### Classifier Backpropagation

In [ ]:
multiLayerNN = MultiLayerNN(X_train, w_hidden_node, w_output_node, var_batch, 
                            var_learning_rate, 1e-6, 
                            momentum = var_momentum, epochs = var_epoch)
multiLayerNN.train(y_train)
print('Matriks weight hidden node:\n', multiLayerNN.w_hidden_node)
print('Matriks weight output node:\n', multiLayerNN.w_output_node)

y_test_res = multiLayerNN.predict(X_test)
print("Hasil Prediksi Train Test: ")
print(y_test_res)

##### Classifier menggunakan keras

In [48]:
model.fit(X_train, to_categorical(y_train, num_classes = 16), batch_size=var_batch, epochs=var_epoch, verbose=1)
prediction = model.predict(X_test)
score = model.evaluate(X_test, to_categorical(y_test, num_classes = 16), verbose=0)
print("Hasil Prediksi Train Test: ")
print(prediction)
print("Hasil Pengukuran Perfomansi Train Test: ")
print(score)

Epoch 1/10
11334/11334 [==============================] - 9s 817us/step - loss: 2.1039 - acc: 0.3855
Epoch 2/10
11334/11334 [==============================] - 10s 874us/step - loss: 2.1045 - acc: 0.3854
Epoch 3/10
11334/11334 [==============================] - 11s 947us/step - loss: 2.1053 - acc: 0.3858
Epoch 4/10
11334/11334 [==============================] - 11s 990us/step - loss: 2.1069 - acc: 0.3857
Epoch 5/10
11334/11334 [==============================] - 13s 1ms/step - loss: 2.0999 - acc: 0.3858
Epoch 6/10
11334/11334 [==============================] - 15s 1ms/step - loss: 2.0925 - acc: 0.3855
Epoch 7/10
11334/11334 [==============================] - 11s 995us/step - loss: 2.0959 - acc: 0.3851
Epoch 8/10
11334/11334 [==============================] - 13s 1ms/step - loss: 2.0963 - acc: 0.3855
Epoch 9/10
11334/11334 [==============================] - 16s 1ms/step - loss: 2.0934 - acc: 0.3872
Epoch 10/10
11334/11334 [==============================] - 18s 2ms/step - loss: 2.0812 - ac

ValueError: Error when checking target: expected dense_12 to have shape (16,) but got array with shape (1,)

#### Implementasi Mini-Batch (Batch_size = jumlah_data)¶

In [ ]:
var_batch = X_train.shape[0]

##### Classifier Backpropagation

In [ ]:
multiLayerNN = MultiLayerNN(X_train, w_hidden_node, w_output_node, var_batch, 
                            var_learning_rate, 1e-6, 
                            momentum = var_momentum, epochs = var_epoch)
multiLayerNN.train(y_train)
print('Matriks weight hidden node:\n', multiLayerNN.w_hidden_node)
print('Matriks weight output node:\n', multiLayerNN.w_output_node)

y_test_res = multiLayerNN.predict(X_test)
print("Hasil Prediksi Train Test: ")
print(y_test_res)

##### Classifier menggunakan keras

In [40]:
model.fit(X_train, to_categorical(y_train, num_classes = 16), batch_size=var_batch, epochs=var_epoch, verbose=1)
score = model.evaluate(X_test, to_categorical(y_test, num_classes = 16), verbose=0)
prediction = model.predict(X_test)
print("Hasil Prediksi Train Test: ")
print(prediction)
print("Hasil Pengukuran Perfomansi Train Test: ")
print(score)

Epoch 1/10
11334/11334 [==============================] - 9s 762us/step - loss: 2.1488 - acc: 0.3855
Epoch 2/10
11334/11334 [==============================] - 9s 773us/step - loss: 2.1222 - acc: 0.3858
Epoch 3/10
11334/11334 [==============================] - 9s 779us/step - loss: 2.1245 - acc: 0.3857
Epoch 4/10
11334/11334 [==============================] - 9s 789us/step - loss: 2.1204 - acc: 0.3864
Epoch 5/10
11334/11334 [==============================] - 9s 800us/step - loss: 2.1379 - acc: 0.3844
Epoch 6/10
11334/11334 [==============================] - 10s 868us/step - loss: 2.1086 - acc: 0.3868
Epoch 7/10
11334/11334 [==============================] - 11s 950us/step - loss: 2.1131 - acc: 0.3873
Epoch 8/10
11334/11334 [==============================] - 16s 1ms/step - loss: 2.1051 - acc: 0.3871
Epoch 9/10
11334/11334 [==============================] - 14s 1ms/step - loss: 2.1051 - acc: 0.3862
Epoch 10/10
11334/11334 [==============================] - 11s 1ms/step - loss: 2.1201 - ac

#### Perbandingan & Analisis Hasil Classifier A dan B



### Pembagian Tugas
1. Alvin Sullivan - 13515048 - 
2. Albertus Djauhari - 13515054 - 
3. Kevin - 13515138 - 